# Sentiment Analysis

In [20]:
#import dependancies
from pathlib import Path
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import hvplot.pandas
from datetime import timedelta , datetime
from config import api_key

In [33]:
#Read in csv files

#$nyse_price_csv = Path("Resources/nyse_prices-split-adjusted.csv")

nyse_sec_csv = Path("Resources/nyse_securities.csv")

spx_csv = Path("Resources/SPX.csv")

#create df's
#nyse_price_df = pd.read_csv(nyse_price_csv)
nyse_sec_df = pd.read_csv(nyse_sec_csv)
spx_df = pd.read_csv(spx_csv)


In [34]:
#Preview NYSE price data
nyse_price_df.head()

NameError: name 'nyse_price_df' is not defined

In [35]:
#Preview NYSE securities data
nyse_sec_df.head()

,Ticker symbol,Security,SEC filings,GICS Sector,GICS Sub Industry,Address of Headquarters,Date first added,CIK
0,MMM,3M Company,reports,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",NaN,66740
1,ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800
2,ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152
3,ACN,Accenture plc,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373
4,ATVI,Activision Blizzard,reports,Information Technology,Home Entertainment Software,"Santa Monica, California",2015-08-31,718877


In [36]:
#Preview NYSE SP500 data
spx_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,1927-12-30,17.660000,17.660000,17.660000,17.660000,17.660000,0
1,1928-01-03,17.760000,17.760000,17.760000,17.760000,17.760000,0
2,1928-01-04,17.719999,17.719999,17.719999,17.719999,17.719999,0
3,1928-01-05,17.549999,17.549999,17.549999,17.549999,17.549999,0
4,1928-01-06,17.660000,17.660000,17.660000,17.660000,17.660000,0


# Data Cleaning

In [37]:
# Lets look for null values in the nyse price dataset
nyse_price_df.isna().sum()
#Lets change symbol column to Ticker Symbol for merging
nyse_price_df["Ticker symbol"] = nyse_price_df["symbol"]
# drop old symbol column
nyse_price_df= nyse_price_df.drop(["symbol"],axis = 1)
#set index to ticker symbol
#nyse_price_df = nyse_price_df.set_index(nyse_price_df["Ticker symbol"])
nyse_price_df.head()

NameError: name 'nyse_price_df' is not defined

In [38]:
# Lets look for null values in the nyse securities dataset
nyse_sec_df.isna().sum() #198 na values in the date first added column

#drop Date first added and Address of Headquarters columns because we won't be needing them

nyse_sec_clean = nyse_sec_df.drop(["Date first added","Address of Headquarters"],axis = 1)
#Preview Cleaned Data Set

nyse_sec_clean.head()


,Ticker symbol,Security,SEC filings,GICS Sector,GICS Sub Industry,CIK
0,MMM,3M Company,reports,Industrials,Industrial Conglomerates,66740
1,ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment,1800
2,ABBV,AbbVie,reports,Health Care,Pharmaceuticals,1551152
3,ACN,Accenture plc,reports,Information Technology,IT Consulting & Other Services,1467373
4,ATVI,Activision Blizzard,reports,Information Technology,Home Entertainment Software,718877


In [39]:
# Lets look for null values in the SP500 dataset
spx_df.isna().sum() ## all good no null values

spx_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,1927-12-30,17.660000,17.660000,17.660000,17.660000,17.660000,0
1,1928-01-03,17.760000,17.760000,17.760000,17.760000,17.760000,0
2,1928-01-04,17.719999,17.719999,17.719999,17.719999,17.719999,0
3,1928-01-05,17.549999,17.549999,17.549999,17.549999,17.549999,0
4,1928-01-06,17.660000,17.660000,17.660000,17.660000,17.660000,0


In [40]:
# Lets merge the NYSE data sets together based on ticker symbol
combined_df = nyse_price_df.merge(nyse_sec_clean, on = "Ticker symbol",how ="inner")

combined_df = combined_df.set_index("Ticker symbol")

combined_df.head()

NameError: name 'nyse_price_df' is not defined

# SP500 Analysis

In [41]:
#Filter from 1928-1960 for Visualization of the first section of years in this dataset.

spx_28_60_df = spx_df[(spx_df["Date"]> "1928-01-01") & (spx_df["Date"]< "1960-01-01")] 
spx_28_60_df["Date"] = pd.to_datetime(spx_28_60_df["Date"])
#Create Line plot of filtered Data
    
spx_28_60_df.hvplot.line(x='Date', y='Adj Close', title=' SP500 1928 -1960', value_label='Average Stock Closing Price',
                              frame_width=700, frame_height=500, group_label='Daily Average')


/var/folders/qn/_ggx04zx5hn5grglx40xmqj80000gn/T/ipykernel_19332/1131391045.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spx_28_60_df["Date"] = pd.to_datetime(spx_28_60_df["Date"])


:Curve   [Date]   (Adj Close)

In [42]:
#Filter from 1960-1990 for Visualization of the first section of years in this dataset.
spx_60_90_df = spx_df[(spx_df["Date"]> "1960-01-01") & (spx_df["Date"]< "1990-01-01")] 
#Create Line plot of filtered Data
spx_60_90_df["Date"] = pd.to_datetime(spx_60_90_df["Date"])
spx_60_90_df.hvplot.line(x='Date', y='Adj Close', title=' SP500 1960 -1990', value_label='Average Stock Closing Price',
                              frame_width=700, frame_height=500, group_label='Daily Average')

/var/folders/qn/_ggx04zx5hn5grglx40xmqj80000gn/T/ipykernel_19332/1561161764.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spx_60_90_df["Date"] = pd.to_datetime(spx_60_90_df["Date"])


:Curve   [Date]   (Adj Close)

In [43]:
#Filter from 1990-2020 for Visualization of the first section of years in this dataset.
spx_90_2020_df = spx_df[(spx_df["Date"]> "1990-01-01") & (spx_df["Date"]< "2020-01-01")] 
#Create Line plot of filtered Data
spx_90_2020_df["Date"] = pd.to_datetime(spx_90_2020_df["Date"])
spx_90_2020_df.hvplot.line(x='Date', y='Adj Close', title=' SP500 1990 -2020', 
                           value_label='Average Stock Closing Price',
                              frame_width=700, frame_height=500, group_label='Daily Average')



/var/folders/qn/_ggx04zx5hn5grglx40xmqj80000gn/T/ipykernel_19332/1572840085.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spx_90_2020_df["Date"] = pd.to_datetime(spx_90_2020_df["Date"])


:Curve   [Date]   (Adj Close)

In [44]:
#Lets Create the Daily Change column
spx_df["Percent Change"] = (spx_df['Adj Close'] - spx_df['Open'])/spx_df['Open']*100
#Preview
spx_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Percent Change
0,1927-12-30,17.660000,17.660000,17.660000,17.660000,17.660000,0,0.0
1,1928-01-03,17.760000,17.760000,17.760000,17.760000,17.760000,0,0.0
2,1928-01-04,17.719999,17.719999,17.719999,17.719999,17.719999,0,0.0
3,1928-01-05,17.549999,17.549999,17.549999,17.549999,17.549999,0,0.0
4,1928-01-06,17.660000,17.660000,17.660000,17.660000,17.660000,0,0.0


In [45]:
#Lets Describe the data to find out what Daily Change Values are considered average
spx_df["Percent Change"].describe() #mean = 0.018129, min = -20.466931, max = 10.789006

count    23323.000000
mean         0.018129
std          0.781987
min        -20.466931
25%         -0.121532
50%          0.000000
75%          0.212481
max         10.789006
Name: Percent Change, dtype: float64

In [46]:
#Lets filter the data down to only show dates when the daily change was above 10
spx_df.loc[spx_df["Percent Change"]>0,:] # There are 7802 values

#Lets go further to filter this range down to a smaller amount of data. Lets try 20 for Daily Change.
spx_df.loc[spx_df["Percent Change"]>3,:] # There are 97 values.

# Lets try to get the data to 10 rows or below to show about 10 of the biggest dates of Daily Change in the history of the SP500.
spx_loss_df = spx_df.loc[spx_df["Percent Change"]>5,:] # There are 16 values.
spx_loss_df


,Date,Open,High,Low,Close,Adj Close,Volume,Percent Change
14994,1987-10-20,225.059998,245.619995,216.460007,236.830002,236.830002,608100000,5.229718
14995,1987-10-21,236.830002,259.269989,236.830002,258.380005,258.380005,449600000,9.099355
17529,1997-10-28,876.989990,923.090027,855.270020,921.849976,921.849976,1202550000,5.115222
17745,1998-09-08,973.890015,1023.460022,973.890015,1023.460022,1023.460022,814800000,5.089898
18331,2001-01-03,1283.270020,1347.760010,1274.619995,1347.560059,1347.560059,1880700000,5.009861
18718,2002-07-24,797.710022,844.320007,775.679993,843.429993,843.429993,2775560000,5.731402
18721,2002-07-29,852.840027,898.960022,852.840027,898.960022,898.960022,1778650000,5.407813
20285,2008-10-13,912.750000,1006.929993,912.750000,1003.349976,1003.349976,7263370000,9.926045
20296,2008-10-28,848.919983,940.510010,845.270020,940.510010,940.510010,7096950000,10.789006
20308,2008-11-13,853.130005,913.010010,818.690002,911.289978,911.289978,7849120000,6.817246


In [ ]:
# The date ranges that appear here are 2008-10-13/10-28 Houseing Crash, 2018-02-06, 2018-12-26 # Not sure, 2020-03 Covid Pandemic

In [47]:
# Now I will look at the top dates per time period charted above. Here is 1928-1960
spx_28_60_df["Percent Change"] = spx_28_60_df["Close"] - spx_28_60_df["Open"]/spx_28_60_df["Open"]
spx_28_60_df["Percent Change"].max() # Daily Change Max was 0. Open and close must not have been calculated in this time peried.

/var/folders/qn/_ggx04zx5hn5grglx40xmqj80000gn/T/ipykernel_19332/1787081445.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spx_28_60_df["Percent Change"] = spx_28_60_df["Close"] - spx_28_60_df["Open"]/spx_28_60_df["Open"]


59.709999

In [48]:
#Here are the biggest Daily Changes from 1960-1990
spx_60_90_df["Percent Change"] = (spx_60_90_df["Close"] - spx_60_90_df["Open"])/spx_60_90_df["Open"]


spx_60_90_df.loc[spx_60_90_df["Percent Change"]>0.05,:]

/var/folders/qn/_ggx04zx5hn5grglx40xmqj80000gn/T/ipykernel_19332/3581588425.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spx_60_90_df["Percent Change"] = (spx_60_90_df["Close"] - spx_60_90_df["Open"])/spx_60_90_df["Open"]


,Date,Open,High,Low,Close,Adj Close,Volume,Percent Change
14994,1987-10-20,225.059998,245.619995,216.460007,236.830002,236.830002,608100000,0.052297
14995,1987-10-21,236.830002,259.269989,236.830002,258.380005,258.380005,449600000,0.090994


In [49]:
#Here are the biggest Daily Changes from 1960-1990
spx_90_2020_df["Percent Change"] = (spx_90_2020_df["Close"] - spx_90_2020_df["Open"])/spx_90_2020_df["Open"]


spx_90_2020_df.loc[spx_90_2020_df["Percent Change"]>0.05,:] # Lots of values greater than 10, had to filter it to .05.

/var/folders/qn/_ggx04zx5hn5grglx40xmqj80000gn/T/ipykernel_19332/996501554.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spx_90_2020_df["Percent Change"] = (spx_90_2020_df["Close"] - spx_90_2020_df["Open"])/spx_90_2020_df["Open"]


,Date,Open,High,Low,Close,Adj Close,Volume,Percent Change
17529,1997-10-28,876.989990,923.090027,855.270020,921.849976,921.849976,1202550000,0.051152
17745,1998-09-08,973.890015,1023.460022,973.890015,1023.460022,1023.460022,814800000,0.050899
18331,2001-01-03,1283.270020,1347.760010,1274.619995,1347.560059,1347.560059,1880700000,0.050099
18718,2002-07-24,797.710022,844.320007,775.679993,843.429993,843.429993,2775560000,0.057314
18721,2002-07-29,852.840027,898.960022,852.840027,898.960022,898.960022,1778650000,0.054078
20285,2008-10-13,912.750000,1006.929993,912.750000,1003.349976,1003.349976,7263370000,0.099260
20296,2008-10-28,848.919983,940.510010,845.270020,940.510010,940.510010,7096950000,0.107890
20308,2008-11-13,853.130005,913.010010,818.690002,911.289978,911.289978,7849120000,0.068172
20314,2008-11-21,755.840027,801.200012,741.020020,800.030029,800.030029,9495900000,0.058465
20315,2008-11-24,801.200012,865.599976,801.200012,851.809998,851.809998,7879440000,0.063168


In [50]:
# Because the events that showed up were very important events in global financial history, I want to actually filter the data down to inclide
#more nuanced changes in value, the kind that would happen more frequently throughout a fiscal year.
spx_90_2020_df.loc[spx_90_2020_df["Percent Change"].between(0.05,0.3),:]   



,Date,Open,High,Low,Close,Adj Close,Volume,Percent Change
17529,1997-10-28,876.989990,923.090027,855.270020,921.849976,921.849976,1202550000,0.051152
17745,1998-09-08,973.890015,1023.460022,973.890015,1023.460022,1023.460022,814800000,0.050899
18331,2001-01-03,1283.270020,1347.760010,1274.619995,1347.560059,1347.560059,1880700000,0.050099
18718,2002-07-24,797.710022,844.320007,775.679993,843.429993,843.429993,2775560000,0.057314
18721,2002-07-29,852.840027,898.960022,852.840027,898.960022,898.960022,1778650000,0.054078
20285,2008-10-13,912.750000,1006.929993,912.750000,1003.349976,1003.349976,7263370000,0.099260
20296,2008-10-28,848.919983,940.510010,845.270020,940.510010,940.510010,7096950000,0.107890
20308,2008-11-13,853.130005,913.010010,818.690002,911.289978,911.289978,7849120000,0.068172
20314,2008-11-21,755.840027,801.200012,741.020020,800.030029,800.030029,9495900000,0.058465
20315,2008-11-24,801.200012,865.599976,801.200012,851.809998,851.809998,7879440000,0.063168


# Application Creation

In [82]:
# We would like our application to take a date input and output a list of headlines from the date selected as well 
#as 5 entries before and after the date selected. 

# We will need to just limit output to what serves our purposes
# Save config information
# use 2002-07-22 as example
# Base url
url = "https://api.nytimes.com/svc/archive/v1/"

# Here is where we will set the year and month as an input values.
year = input("What year would you like to search: ")
month =input("What month would you like to search: ")


# Build query URL - change url to just look for nyt api search
query_url = url + year + "/" + month + ".json?api-key=" + api_key
# Get NYT headline data
NYT_response = requests.get(query_url).json()

What year would you like to search: 2002
What month would you like to search: 7


In [85]:
#initialize column lists for NYT_df variables of interest
headline=[]
news_desk = []
section = []
date = []
snippet = []
web_url = []

#Here we write the for loop to loop through the api request to find specific information from each article.
for i in range(len(NYT_response['response']['docs'])):
    headline.append(NYT_response['response']['docs'][i]['headline']['print_headline'])
    news_desk.append(NYT_response['response']['docs'][i]['news_desk'])
    section.append(NYT_response['response']['docs'][i]['section_name'])
    date.append(NYT_response['response']['docs'][i]['pub_date'])
    snippet.append(NYT_response['response']['docs'][i]['snippet'])
    web_url.append(NYT_response['response']['docs'][i]['web_url'])
    
    

In [88]:
#create NYT dictionary.
NYT_dict = {
    "headline": headline,
    "News Desk": news_desk,
    "Section": section,
    "Date": date,
    "Snippet": snippet,
    "web_url": web_url
}
# Here we created a data frame from the dictionary
NYT_df = pd.DataFrame(NYT_dict)
#Here we split the Date and time to remove the timecode portion of the date column. 
NYT_df[["Date","Time"]] = NYT_df.Date.str.split("T",expand = True)
# input pub date to further filter the data to only the date we are interested in.
day = input("What day would you like to search: ")
#Build pub_date string
pub_date = f'{year}-{month}-{day}'
#Formatting pub_date
pub_date = datetime.strptime(pub_date,'%Y-%m-%d')
#drop Time column to clean up output dataset.
NYT_df = NYT_df.drop(["Time"], axis = 1) 



What day would you like to search: 22


In [96]:
#filter data by pub_date
NYT_df["Date"] = pd.to_datetime(NYT_df["Date"])
# Here we use the loc function to filter our new data set to only show the pub date we are interested in.
NYT_df_new = NYT_df.loc[NYT_df["Date"]==pub_date]
#We found several articles had blank headlines. Here we replace blank headlines with Nan.
NYT_df_new["headline"].replace("",np.nan,inplace = True)
# Now we drop all values with Nan headlines as they will not be valueble in our analysis.
NYT_df_new.dropna(inplace= True)
# Now we are filtering the data to only show the Business desk news from the selected date.
NYT_search = NYT_df_new.loc[NYT_df_new["News Desk"]== "Business",:]

NYT_search.iloc[3]

/var/folders/qn/_ggx04zx5hn5grglx40xmqj80000gn/T/ipykernel_19332/2583150582.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  NYT_df_new["headline"].replace("",np.nan,inplace = True)
/var/folders/qn/_ggx04zx5hn5grglx40xmqj80000gn/T/ipykernel_19332/2583150582.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  NYT_df_new.dropna(inplace= True)


headline                            The Week's Economic Events
News Desk                                             Business
Section                                           Business Day
Date                                       2002-07-22 00:00:00
Snippet                                            MONDAYNone.
web_url      https://www.nytimes.com/2002/07/22/business/th...
Name: 6068, dtype: object